# 6章 デプロイ

## 6.1 継続的インテグレーションとは

* CIの主な目的はすべてを互いに常に同期させること
  * 実際には新たにチェックインされたコードが既存コードと適切に統合（インテグレーション）されるようにすること


* 理想的には一度だけのビルドで、そのバージョンの他のコードのすべてのデプロイで再利用したい


* CIの利点
  * コードの品質に関して高速なフィードバック
  * バイナリ作成物の作成を自動化
  * 必要に応じて成果物を再作成可能
  * 成果物からコードへのある程度のトレーサビリティ

## 6.1.1 実際にCIを行っているか

* CIを行っていると言っているにもかかわらず実際には全く行っていない多くのチーム
  * CIツールの利用とCIのプラクティスを混同している


* CIとは何かを本当に理解しているかを調べるための3つの質問 (Jez Humble)
  * 1日に一度はメインラインにチェックインしていますか。
    * メインブランチに頻繁にマージすること
  * 変更を検証するテストスイートがありますか。
  * ビルドが壊れたときに、それを修正するのがチームの最優先事項でしょうか。
    * 壊れたビルドの修正とは関係ない変更を増やしていくと、ビルドを修正するためにかかる時間が増えていく

## 6.2 継続的インテグレーションとマイクロサービスへのマッピング

* CIビルドを個々のマイクロサービスにどのようにマッピングするか
  * 残りのサービスとは独立して、1つのサービスに変更を行いそれをデプロイできるようにしたい


* 勧められないが最も簡単な方法はすべてをひとまとめにすること
  * 全コードを格納する巨大なリポジトリと1つのビルド
  * 同時リリースを行う場合にはうまく機能する
    * 同時リリースは一般的には絶対避けるべき
    * プロジェクトの初期段階など短期的には理にかなっていることもある
  * 欠点
    * 1つのサービスを変更すると他のサービスもすべて検証されてビルドされる
    * 関係のないビルドのために時間が大幅にかかる
    * どれをデプロイすべきか判断しにくい
    * ビルドを壊してしまったときに他のサービスの変更ができなくなる
    * ビルドの責任を誰が負うか
  * この方法のバリエーションは複数のCIビルドをソースツリーの一部に対応付ける方法
    * 複数サービスのソースコードを一度にチェックインする習慣がとてもつきやすくなり、サービスを結合する変更にも陥りやすくなる (?)


* 代替手段: マイクロサービスごとに1つのCIビルドを持つこと

## 6.3 ビルドパイプラインと継続的デリバリ

* ビルドには複数のステージがあると便利
  * 多数の高速でスコープの小さいテストと少数の低速でスコープの大きいテスト
  * 高速のテストで失敗したら低速のテストをやる意味はあまりない
  * ビルドパイプラインを作成する


* 継続的デリバリ
  * 手動と自動の両方でソフトウェアが通過しなければいけないあらゆるステージをモデル化
  * CIツールをハックし拡張してCDを実行するのではなく、CDを第一級の概念として採用しているツールが必要
  * 手動のユーザ受け入れテスト(UAT: User Acceptance Testing)もCDのモデルの中に組み込む
    * 手動でテストを行ったあと、後続のステージに進むトリガーを手動で設定
  * ソフトウェアの本番環境までの経路全体をモデル化すると、 ... リリースにかかる時間も大幅に短縮できます。

## 6.3.1 避けられない例外

* 基本的には「ビルドごとに1つのマイクロサービス」の手法を目指すべき
* 例外
  * 新規のプロジェクトではすべてのサービスを1つのビルドにすることは理にかなっている
    * その場合は当然すべてのサービスが一括リリースになる
  * サービス境界を割り出すために大きな変動がある可能性が高い
  * サービスAPIが安定したらビルドを分割する移行ステージが必要

## 6.4 プラットフォーム固有の成果物

* 成果物の作成とインストールをサポートするツール
  * Rubyのgem、JavaのJARファイル、WARファイル、PythonのEgg
* 構成管理ツール
  * Puppet, Chef, Ansible
  * 成果物を使って実行するために必要な他のツールをインストールして構成するもの
  * RubyやPythonのアプリケーションではApache, nginx内で動作するプロセスマネージャが必要


* さまざまな技術を組み合わせて使うときはデプロイが困難になる
  * 構成管理ツールを使った自動化がデプロイ機構の違いを隠すのに役立つ

## 6.5 OS成果物

* OS固有の成果物
  * Red Hat, CentOSのRPM、Ubuntuのdebパッケージ、WindowsのMSI
  * デプロイの観点から見て基盤となる技術を気に掛けなくてよい
  * Linuxプラットフォームではパッケージの依存関係を定義でき、自動的にインストール可能


* パッケージ作成が困難
  * LinuxではFPMパッケージマネージャツール
  * WindowsではMSIインストーラは機能的に不十分
    * Chocolatey NuGetがLinuxのパッケージマネージャに似ている


* 複数の異なるOSへのデプロイが困難

## 6.6 カスタムイメージ

* 自動構成管理システム(Puppet, Chef, Ansible)は実行するのに時間がかかる
  * マシンのプロビジョニングに数分、Javaアプリケーションを実行するためにOracle JVMをインストールに数分
  * さらにOS統計情報を取得するためにcollectd、ログ集約にLogstash、監視のためにNagios
  * 必要なものが増え、依存関係が増えるごとにかかる時間は長くなっていく
  * 高速なフィードバックを提供するという観点で深刻な問題となる


* システムがゼロダウンタイムデプロイを許していない場合 (?)
  * ダウンタイムがあってもよいということ?


* 仮想マシンイメージの作成
  * 起動時間の短縮
  * 依存関係を含めた仮想マシンイメージを作成しておく（カスタムイメージ）
  * カスタムイメージからインスタンスを起動し、サービスの最新バージョンをインストールするだけ
  * あらかじめイメージに含めておく依存関係が変更にならない限り、同じカスタムイメージを使い続けることができる


* カスタムイメージの欠点
  * イメージの作成に時間がかかる
  * イメージのサイズが大きくなる（コンテナ技術によって一部回避可能）


* イメージを構築するために必要なツールがプラットフォームごとに異なるという課題
  * VMwareイメージ、AWS AMI、Vagrantイメージ、Rackspaceイメージの構築はそれぞれ異なる
  * Packerは同じ構成でさまざまなプラットフォーム向けのイメージを作成できる

## 6.6.1 成果物としてのイメージ

* 依存関係だけではなくサービスもイメージに含める
* イメージを使う側からするとサービスがどのような技術スタック（JavaなのかRubyなのか）で動いているかは興味がなく適切に機能するかどうかだけ

## 6.6.2 イミュータブルサーバ

* 稼働するサービスのための構成をすべてソースコードとして管理
* サービスや環境全体を再現可能になる
* 構成ドリフト: ソース管理とは別に手動で変更を加えること

## 6.7 環境

* 環境はさまざま
  * 低速なテスト環境、UAT環境、性能試験環境、本番環境
  * ステージが進むごとに本番環境に近づけるようにし、環境の違いに関連する問題を早く把握する
  * ただし擬似本番環境を用意するのは大変なのでバランスを考える必要がある

## 6.8 サービス構成

* 環境ごとの特有な設定は別に管理する
  * プロパティファイル、インストールプロセスに渡すパラメータ

## 6.9 サービスからホストへのマッピング

* 一般的な単位としてのホスト
* ホストごとにいくつのサービスを持つべきか

## 6.9.1 ホストごとに複数のサービス

* 良い点
  * ホストの管理が簡単になる


* 悪い点
  * 監視が困難（他のサービスと分離して調べることが難しい）
  * デプロイが困難（他のサービスに配慮したデプロイが必要）
  * サービスをチームごとに分担していた場合、ホストを誰が管理するのか
  * スケーリングを必要とするサービスのデプロイが困難

## 6.9.2 アプリケーションコンテナ

* 良い点
  * クラスタリングサポートや監視ツール
  * 言語ランタイムのオーバーヘッド削減


* 悪い点
  * 技術選択が制限される
  * クラスタを管理して共有インメモリセッション状態をサポートする機能は、サービスのスケールアウトの際に課題がある
  * コンテナの多くは起動に時間がかかる
  * アプリケーションのライフサイクル管理で、単なるJVMの再起動よりも複雑になる場合がある


* 自己完結型のデプロイ可能なマイクロサービス
  * .NETのNancy、JavaのJetty組み込みコンテナ

## 6.9.3 ホストごとに1つのサービス

* 良い点
  * 複数のサービスが存在することによる副作用の回避
  * 単一障害点を減らせる
  * 他のサービスと独立してスケーリングできる


* 悪い点
  * ホストが増えてくるとコスト上の問題

## 6.9.4 PaaS

* warやgemなど技術固有の成果物に依存し、成果物を自動的にプロビジョニングして実行
* 標準が離れているほどPaaSのソリューションはうまくいきにくい
  * 修正のための内部調査に関してあまり制御できない
  * 優れたPaaSソリューションは多くのことに対処する

## 6.10 自動化

* サーバが倍になれば作業も倍になるが、ホストの制御とサービスのデプロイが自動化されていれば、ホスト追加のコストは増えていかない
* ホストの数を少なくしていたとしても、デプロイするサービス、監視するサービス、収集するログは複数あり複雑なため自動化は不可欠
* 開発者が自分でプロビジョニングできるようにしておくことは、開発者の生産性を保つことになる

## 6.10.1 自動化の威力に関する2つのケーススタディ

* RealEstate.com.auの例
* Giltの例

## 6.11 物理から仮想へ

## 6.11.1 従来の仮想化

* ハイパーバイザ
  * ホストOSの上でハイパーバイザが動く
  * CPUやメモリといったリソースを仮想ホストから物理ホストへマッピングする
  * 制御レイヤとして仮想マシンを操作する
  * ハイパーバイザ自体が動作するためのリソースが必要になってしまう

## 6.11.2 Vagrant

* 開発やテストとしてノートPC上に仮想クラウドを提供
* テキストファイルで一連のVM、ネットワーキングを定義でき、チームで共有できる


* 複数のVMを立ち上げるのは平均的なノートPCでは困難
* システム全体を上げられないとなると、一部をスタブ化するなど開発やテストを行いやすくするためのタスクが増える

## 6.11.3 Linuxコンテナ

* VMで空間を分割する替わりにLinuxコンテナでプロセス空間を分割する
* ハイパーバイザが不要
* コンテナごとに異なるOSディストリビューションを実行できるが、カーネルは共有できなければいけない
* プロビジョニングがはるかに高速


* 仮想化と一緒に使用することも可能
  * AWS EC2の上でLXCコンテナを実行


* コンテナで動作するサービスへのルーティングが必要
  * ハイパーバイザが行っていたこと

## 6.11.4 Docker

* コンテナのプロビジョニングを管理
* ネットワーキング問題の一部に対処


* Dockerのアプリ抽象化
  * サービスのビルドでDockerアプリケーションのイメージを作成
  * Dockerレジストリにイメージを格納して使用


* CoreOS
  * debやRPMといったパッケージマネージャを使うのではなく、すべてのソフトウェアをDockerアプリとしてインストール


* 複数ホストで動かすDocker
  * スケジューリングレイヤが必要: KubernetesやCoreOSのクラスタ技術
  * スケジューリングレイヤを備えたDockerはIaaSとPaaSの間、CaaS (Container as a Service)と呼ばれる

## 6.12 デプロイのインタフェース

* デプロイのための統一インタフェースが必要
* PythonライブラリFabricの例

## 6.12.1 環境定義

* AWSの環境定義をするYAMLファイルの例
* Terraformも近い思想のツール

## 6.13 まとめ

* サービスを他のサービスと独立してリリースできること
* マイクロサービスごとに1つのリポジトリ、1つのCIビルド
* ホスト/コンテナごとに1つのサービス
* セルフサービスデプロイができるようなツールを用意すること